In [ ]:
#default_exp unstructured2markup

In [ ]:
#no_test

#converts txt version of Genkins transcription into XML formatted according to SH specs, originally built for DSI entity extraction project

txt_path = input("Enter absolute path to txt version of trancription: ")
volume_title = input("Enter volume title: ")
volume_id = input("Enter volume identifier: ")
output_path = input("Enter file name for output: ")

volume = open(txt_path, 'r', encoding="utf-8")
output = open(output_path + ".xml", 'w', encoding="utf-8")

def writeHeader(title, out, id=None):
    out.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")
    out.write("<!--" + title + "-->\n")
    out.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
    out.write("<volume>\n")
    out.write("<volumeTitle>" + title + "</volumeTitle>\n")
    if id != None:
        out.write("<volumeIdentifier>" + id + "</volumeIdentifier>")
    else:
        out.write("<volumeIdentifier />\n")
    out.write("<volumeGuid />\n")

def writeFooter(out):
    out.write("</text>\n")
    out.write("</item>\n")
    out.write("</volume>\n")
    out.write("</ssda>\n")

writeHeader(volume_title, output, volume_id)

in_entry = False
first_folio = True
folio_no = 1001

for line in volume:
    if ("Fol" in line) and (first_folio == True):
        first_folio = False
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and (in_entry == False):
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and in_entry:
        output.write("</entry>\n")
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
        output.write("<entry>\n")
    elif line[:3] == "[ma" and (in_entry == False):
        output.write("<entry>\n")
        output.write(line)
        in_entry = True
    elif in_entry and line[:3] == "[si":
        in_entry = False
        output.write(line)
        output.write("</entry>\n")
    elif line[:3] == "[ma":
        output.write("</entry>\n")
        output.write("<entry>\n")
        output.write(line)
    elif in_entry:
        line = line.replace('<', '')
        line = line.replace('>', '')
        output.write(line)

writeFooter(output)

volume.close()
output.close()
